# Analisis del DataSet Adult

## Exportación y Carga del Dataset Adult

In [7]:
import pandas as pd

# URL del dataset en el repositorio de UCI
url = "adult/adult.data"
columnas = ["age", "workclass", "fnlwgt", "education", "education-num", "marital-status",
            "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss",
            "hours-per-week", "native-country", "income"]
# Leer CSV y limpiar espacios / valores faltantes
df = pd.read_csv(url, names=columnas, na_values="?", skipinitialspace=True)

## Exploración visual de los datos

In [10]:
# Cantidad de filas y columnas del DataFrame
print("Forma del DataFrame:", df.shape)

# Identificar columnas numéricas y categóricas
disc_cols = df.select_dtypes(include=["int64"]).columns.tolist()
cont_cols = df.select_dtypes(include=['float64']).columns.tolist()
cat_cols = df.select_dtypes(include=['object']).columns.tolist()

print("Columnas cuantitativas discreta:", disc_cols)
print("Columnas cuantitativas continuas:", cont_cols)
print("Columnas categóricas:", cat_cols)

# Ver las primeras filas
df.head()

Forma del DataFrame: (32561, 15)
Columnas cuantitativas discreta: ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
Columnas cuantitativas continuas: []
Columnas categóricas: ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country', 'income']


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


## Tratamiento de valores nulos

In [11]:
print("\nConteo de valores faltantes (NaN) por columna:")
df.isnull().sum()


Conteo de valores faltantes (NaN) por columna:


age                  0
workclass         1836
fnlwgt               0
education            0
education-num        0
marital-status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital-gain         0
capital-loss         0
hours-per-week       0
native-country     583
income               0
dtype: int64

In [ ]:
# WorkClass es de tipo Categórico


# Ocuppation es de tipo Categórico

# Native Country es de tipo Categórico